Steps:
1. Make a beautiful soup object from homepage to extract the links of postable locations
2. Get zipcodes for each location
3. Create a dataframe with location, link, zipcode

In [2]:
def makesoup(url):
    import requests, bs4, re
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, "lxml")
    return(soup)

In [31]:
#Create bs object from clist homepage in order to extract the links of all postable locations
url = 'https://newyork.craigslist.org/'
soup = makesoup(url)
link_info = soup.find('li', {'class':'s'} )

In [43]:
# Clean up the link_info using regex to get rid of everything but the urls and locations
import re
pat = re.compile('"\/\/(\w+\.\w+\.org\/">\w+\s?\w+)')
m = re.findall(pat,str(link_info))
with open('clistlinks.txt', 'w') as wf:
    for item in m:
        wf.write(item)
        wf.write('\n')

In [4]:
#Form lists for the urls and locations
urls = []
locs = []
with open('clistlinks.txt', 'r') as rf:
    for line in rf.readlines():
        l=line.split('"')
        urls.append(l[0])
        locs.append(l[1].replace('"',"").replace('>','').replace('\n',''))
#can use mult chars to split on

In [112]:
#Add prefix to urls 
links=[]
for item in urls:
    new="https://"+item
    links.append(new)

In [20]:
# Create dataframe with the locations and the links
import pandas as pd
df = pd.DataFrame(links,index=locs,columns=['link'])

In [8]:
# Get zip codes for each location using selenium and create dictionary connecting zip codes to location
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver = webdriver.Chrome()
d={}
for elem in locs:
    driver.get("http://zipcode.org/zip_code_search")
    e = driver.find_element_by_name("txtTownCity")
    e.clear()
    e.send_keys(elem)
    e.send_keys(Keys.RETURN)
    try:
        content = driver.find_element_by_class_name('Search_Results_Link_Text')
        zipcode = content.get_property("text")
        d[elem]=zipcode
    except:
        d[elem]='unknown'
driver.close()
#update to include extra landing page, use case/switch

In [58]:
# Group together those that failed and try again using google
fails=[]
for key,val in d.items():
    if val=='unknown':
        fails.append(key)
#redo as one line list comprehension

for elem in fails:
    driver = webdriver.Chrome()
    driver.get("http://www.google.com")
    e = driver.find_element_by_name("q")
    e.clear()
    e.send_keys(elem+' zip codes')
    e.send_keys(Keys.RETURN)

In [47]:
#See if there is any other zips still missing 
for elem in locs:
    if elem not in d:
        print(elem)

binghamton
boston
catskills
eastern ct
north jersey
south coast


In [55]:
#Add missing zips to dictionary by hand
d['binghamton']='13901'
d['catskills']='12414'
d['boston']='02110'
d['north jersey']='07024'
d['eastern ct']='06226'
d['south coast']='02790'

In [104]:
#Ensure that the length of dictionary matches the number of rows in the dataframe
len(d)==df.shape[0]

True

In [96]:
#Add zip codes to dataframe
df['zip_codes']=d.values()
df.index.name = 'location'
df.columns = ['url','zip_code']

In [68]:
#Create dictionary assigning each location as either hs or college and add that info to dataframe
colleges=['binghamton','boston','hartford','ithaca','oneonta','syracuse','utica','western mass']
hs=[x for x in d if x not in colleges]
col_hs={}
for elem in d:
    if elem in colleges:
        col_hs[elem]="col"
    else:
        col_hs[elem]="hs"
df['ad_type']=col_hs.values()

In [108]:
#save dataframe to pickle to easily restore later
df.to_pickle('clist_df')

Next steps:
3. Use Selenium to post an ad to a particular location using a particular username and password
4. Post several ads in multiple locations by looping through various username/password combinations